# Salesforce Sales Ops 8-1-23
Harry Hill

### Outstanding hotfixes:
- 110/4532 rows not given a Contact Level; investigate final gaps manually and add into regex's
- make sure none of the Department labels overwrite each other
- manually check for Accounting and Legal titles (0 came up)
- some phone numbers in XXX-XXX-XXXX format; want them transformed?

## Import dataset

In [766]:
import pandas as pd
import numpy as np
import re
from titlecase import titlecase

file = '08-1-23 Sales Ops NEW for Harry.xlsx'

df = pd.read_excel(file)

df.head(50)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Manager, Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,NaN,NaN
1,100 Thieves,Kelsey,Schultz,"Director, Head of Sales Operations and Activat...",k.schultz@100thieves.com,(424) 298-0196,NaN,NaN,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,Chief Technology Officer,matt@10kview.com,NaN,NaN,NaN,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,NaN,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,NaN,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,NaN,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,NaN,NaN
8,1Password,Emily,Payne,Sales Enablement Manager,emily.payne@1password.com,(888) 710-9976,NaN,NaN,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Manager, Sales Operations",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,NaN,NaN


## 1. Convert 'Title'

In [767]:
# make a copy dataframe to edit
df2 = df.copy()
df2.head()

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Manager, Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,NaN,NaN
1,100 Thieves,Kelsey,Schultz,"Director, Head of Sales Operations and Activat...",k.schultz@100thieves.com,(424) 298-0196,NaN,NaN,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,Chief Technology Officer,matt@10kview.com,NaN,NaN,NaN,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,NaN,NaN


In [768]:
# replace nulls in 'Title' with '-'
df2['Title'] = df2['Title'].fillna('-')
df2[df2['Title'].isnull()]


,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department


### Make 'Title' column titlecase
- i.e. coordinator -> Coordinator

In [769]:
df2['Title'] = df2['Title'].astype(str).apply(lambda x: titlecase(x))
df2.head(50)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Manager, Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,NaN,NaN
1,100 Thieves,Kelsey,Schultz,"Director, Head of Sales Operations and Activat...",k.schultz@100thieves.com,(424) 298-0196,NaN,NaN,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,Chief Technology Officer,matt@10kview.com,NaN,NaN,NaN,NaN
4,10K,Jared,Miller,Coo,jared@10kview.com,NaN,NaN,NaN,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,NaN,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,NaN,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,NaN,NaN
8,1Password,Emily,Payne,Sales Enablement Manager,emily.payne@1password.com,(888) 710-9976,NaN,NaN,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Manager, Sales Operations",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,NaN,NaN


### Make all acronyms in 'Title' column uppercase
- must work on acronyms in titlecase i.e. Svp from previous step

In [770]:
# list of acronyms that should be uppercase from
acronym_list = ['CEO', 'CFO', 'COO', 'CSO', 'CRO', 'CMO', 'CTO', 'SVP', 'EVP', 'AVP', 'VP', 'AP', 'A/P', 'AR', 'A/R', 'AE', 'QA']

# Create a regular expression pattern that matches any acronyms in the list
pattern = r'\b(?:' + '|'.join(re.escape(acronym) for acronym in acronym_list) + r')\b'

# Function to convert matches to uppercase
def make_matches_uppercase(match):
    return match.group().upper()

# Apply the regular expression and conversion function to the 'Title' column
# re.sup expects a string, so converted column
df2['Title'] = df2['Title'].astype(str).apply(lambda x: re.sub(pattern, make_matches_uppercase, x, flags=re.IGNORECASE))

df2.head(50)



,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Manager, Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,NaN,NaN
1,100 Thieves,Kelsey,Schultz,"Director, Head of Sales Operations and Activat...",k.schultz@100thieves.com,(424) 298-0196,NaN,NaN,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,Chief Technology Officer,matt@10kview.com,NaN,NaN,NaN,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,NaN,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,NaN,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,NaN,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,NaN,NaN
8,1Password,Emily,Payne,Sales Enablement Manager,emily.payne@1password.com,(888) 710-9976,NaN,NaN,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Manager, Sales Operations",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,NaN,NaN


### (a) C-level Execs
- Chief Executive Officer to CEO
- Chief Financial Officer to CFO
- Chief Operating Officer to COO
- Chief Sales Officer to CSO
- Chief Revenue Officer to CRO
- Chief Marketing Officer to CMO
- Chief Technology Officer to CTO

In [771]:
# find & replace full-length strings with abbreviations, ignoring case
df2['Title'] = df2['Title'].str.replace(pat='Chief Executive Officer', repl = 'CEO', flags = re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Chief Financial Officer', repl = 'CFO', flags = re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Chief Operating Officer', repl = 'COO', flags = re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Chief Sales Officer', repl = 'CSO', flags = re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Chief Revenue Officer', repl = 'CRO', flags = re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Chief Maerketing Officer', repl = 'CMO', flags = re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Chief Technology Officer', repl = 'CTO', flags = re.IGNORECASE, regex = True)
df2.head(50)


,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Manager, Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,NaN,NaN
1,100 Thieves,Kelsey,Schultz,"Director, Head of Sales Operations and Activat...",k.schultz@100thieves.com,(424) 298-0196,NaN,NaN,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,CTO,matt@10kview.com,NaN,NaN,NaN,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,NaN,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,NaN,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,NaN,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,NaN,NaN
8,1Password,Emily,Payne,Sales Enablement Manager,emily.payne@1password.com,(888) 710-9976,NaN,NaN,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Manager, Sales Operations",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,NaN,NaN


### (b) Management
- Senior Vice President to SVP
- Executive Vice President to EVP
- Assistant Vice President to AVP
- Associate Vice President to AVP
- Vice President to VP
- Financial to Fin’l
- Senior to Sr.
- Director to Dir.
- Manager|Management to Mgr.
- Corporate to Corp.
- Division to Div.
- Executive to Exec. 

In [772]:
# find & replace full-length strings with abbreviations, ignoring case
df2['Title'] = df2['Title'].str.replace(pat='Senior Vice President', repl = 'SVP', flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Executive Vice President', repl = 'EVP', flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Assistant Vice President', repl = 'AVP', flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Associate Vice President', repl = 'AVP', flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Vice President', repl = 'VP', flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat='Financial', repl = "Fin'l", flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat="Senior", repl = "Sr.", flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat="Director", repl = "Dir.", flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat="Manager|Management", repl = "Mgr.", flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat="Corporate", repl = "Corp.", flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat="Division", repl = "Div.", flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat="Executive", repl = "Exec.", flags=re.IGNORECASE, regex = True)
df2.head(50)



,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Mgr., Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,NaN,NaN
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Operations and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,NaN,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,CTO,matt@10kview.com,NaN,NaN,NaN,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,NaN,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,NaN,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,NaN,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,NaN,NaN
8,1Password,Emily,Payne,Sales Enablement Mgr.,emily.payne@1password.com,(888) 710-9976,NaN,NaN,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Operations",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,NaN,NaN


### (c) General Conversions
- Associate to Assoc.
- Professional to Prof’l
- Sales Operations to Sales Ops
- Quality Assurance to QA

In [773]:
# find & replace full-length strings with abbreviations, ignoring case
df2['Title'] = df2['Title'].str.replace(pat="Associate|Assoc", repl = "Assoc.", flags=re.IGNORECASE, regex = True, )
df2['Title'] = df2['Title'].str.replace(pat="Professional", repl = "Prof'l.", flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat="Sales Operations", repl = "Sales Ops", flags=re.IGNORECASE, regex = True)
df2['Title'] = df2['Title'].str.replace(pat="Quality Assurance", repl = "QA", flags=re.IGNORECASE, regex = True)
df2.head(50)



,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Mgr., Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,NaN,NaN
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Ops and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,NaN,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,CTO,matt@10kview.com,NaN,NaN,NaN,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,NaN,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,NaN,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,NaN,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,NaN,NaN
8,1Password,Emily,Payne,Sales Enablement Mgr.,emily.payne@1password.com,(888) 710-9976,NaN,NaN,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Ops",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,NaN,NaN


## 2. Categorize 'Contact Level'

### (a) Staff
- Analyst | Analytics
- Assistant (= last word of title, not Assistant Director, VP, etc.)
- Associate | Assoc. (= last word of title, not Associate Director, VP, etc.)
- Executive Assistant
- Architect
- Accounts Payable, AP, A/P as sole text in title
- Accounts Receivable, AR, A/R as sole text in title
- Administrator, Admin, Administrative
- Coordinator
- Representative
- Clerk
- Account Executive
- AE
- Account Manager
- Specialist
- Staff
- Lead
- Receptionist
- Sales Ops as sole text in title
- Sales Ops Assoc.
- Sales Ops' as end text
- Salesforce Assoc.

- 'Developer'
- 'Support'
- 'Engineer'
- 'Planner / Planning'
- 'Integration'
- 'Strategist'
- 'Marketer'
- 'Instructor'
- 'Trainer'
- 'Advisor'
- 'Trainee'
- 'QA'
- 'Scrum Master'
- 'Recruiter / Recruitment'
- 'Service Operations'
- 'Expert'
- 'Specialist'
- 'Coach'
- 'Admin'

In [774]:
# create condition for containing any of the Staff keywords
staff = (df2['Title'].str.contains(
                                  'Developer|'
                                  'Support|'
                                  'Engineer|'
                                  'Planner|Planning|'
                                  'Integration|'
                                  'Strategist|'
                                  'Marketer|'
                                  'Instructor|'
                                  'Trainer|'
                                  'Advisor|'
                                  'Trainee|'
                                  'QA|'
                                  'Scrum Master|'
                                  'Recruiter|Recruitment|'
                                  'Service Operations|'
                                  'Expert|'
                                  'Specialist|'
                                  'Coach|'
                                  'Admin'
                                  ,na=False, regex=True) |
         df2['Title'].str.contains(r"(?:Analyst|Sales Ops Assoc\.|Salesforce Assoc\.|Analytics|Assistant\n|Assoc\.\n|Executive|Assistant|Architect|Dir\.|\nAccounts Payable\n|\nAP\n|\nA/P\n|\nAccounts Receivable$|\nAR\n|\nA/R\n|Administrator|Admin|Administrative|Coordinator|Representative|Clerk|Account Executive|AE|Account Manager|Specialist|Staff|Lead|Receptionist|Sales Ops$)",na=False, regex=True)
        )

# apply the condition to filter the rows and then set the 'Contact Level' column to 'Staff'
df2.loc[staff, 'Contact Level'] = "Staff"

df2[df2['Contact Level']=='Staff'].head(50)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Ops and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,Staff,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,Staff,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,Staff,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Ops",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,Staff,NaN
10,1WorldSync,Renee,Gerami,Dir. of Technology Planning & Delivery,rgerami@1worldsync.com,(866) 280-4013,NaN,Staff,NaN
15,3STEP Sports,Deion,Wilson,Sales Ops Specialist,deion.wilson@threestep.com,(781) 722-0338,NaN,Staff,NaN
16,4Refuel Canada,Victor,Chatellier,Sales and Operations Coordinator,vchatellier@4refuel.com,+1 888-473-3835,NaN,Staff,NaN
17,4Refuel Canada,Goli,Golzari,Sales and Operations Coordinator,ggolzari@4refuel.com,+1 888-473-3835,NaN,Staff,NaN
20,6sense,Ankit,Goyal,Sr. Integration Mgr.,ankit.goyal@6sense.com,+1 415-305-5951,NaN,Staff,NaN
21,6Street Digital,Johnson,Tai,Salesforce Developer,johnson.tai@6st.co,+1(833) 242-6439,NaN,Staff,NaN


### (b) Management
- SVP
- EVP
- AVP
- VP
- Controller
- Dir.
- Mgr. (except Account Mgr.)
- Supervisor
- Division
- Executive
- Head
- Branch Mgr.
- Partner

In [775]:
# filter dataframe by Management titles, then set the 'Contact Level' columns equal to 'Management'
# SYNTAX: df.loc[df[‘filter column’] condition, ‘target column name’] = ‘target column value if condition is met’
# negative lookbehind assertion to exclude Account Mgr.
# end of line backslashes to escape line created wrap issues for easy reading, used quotes for each line instead
df2.loc[df2['Title'].str.contains(
                                  'SVP|EVP|AVP|VP|'
                                  'Controller|'
                                  'Dir\.|'
                                  '(?<!Account )Mgr\.|'
                                  'Supervisor|'
                                  'Division|'
                                  'Executive|'
                                  'Head|'
                                  'Branch Mgr\.|'
                                  'Partner',
                                  na=False, regex=True),'Contact Level'] = 'Management'

df2.head(50)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Mgr., Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,Management,NaN
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Ops and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,Management,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,CTO,matt@10kview.com,NaN,NaN,NaN,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,NaN,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,Management,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,Staff,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,Staff,NaN
8,1Password,Emily,Payne,Sales Enablement Mgr.,emily.payne@1password.com,(888) 710-9976,NaN,Management,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Ops",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,Management,NaN


### (c) CXO
- CFO
- COO
- CAO
- CSO
- CRO
- CMO
- CTO
- titles containing Board, Chair

In [776]:
# filter dataframe by Chief Officer titles, then set the 'Contact Level' columns equal to 'CXO'
# SYNTAX: df.loc[df[‘filter column’] condition, ‘target column name’] = ‘target column value if condition is met’
# backslashes for splitting expressions into multiple lines
# ADDED 'CTO'
df2.loc[df2['Title'].str.contains(\
                                              'CFO|COO|CAO|CSO|CRO|CMO|CTO|\bBoard\b|\bChair\b',\
                                              na=False, regex=True),'Contact Level']\
                                              = 'CXO'

df2.head(50)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Mgr., Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,Management,NaN
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Ops and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,Management,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,NaN,NaN
3,10K,Matthew,Lamb,CTO,matt@10kview.com,NaN,NaN,CXO,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,CXO,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,Management,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,Staff,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,Staff,NaN
8,1Password,Emily,Payne,Sales Enablement Mgr.,emily.payne@1password.com,(888) 710-9976,NaN,Management,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Ops",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,Management,NaN


### (d) Executive
- CEO 
- President
- Founder
- Co-founder
- Owner

In [777]:
# filter dataframe by Executive titles, then set the 'Contact Level' columns equal to 'Executive'
# SYNTAX: df.loc[df[‘filter column’] condition, ‘target column name’] = ‘target column value if condition is met’
df2.loc[df2['Title'].str.contains('CEO|President|Founder|Co-Founder|Co-founder|Owner', na=False),'Contact Level'] = 'Executive'

df2.head(50)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Mgr., Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,Management,NaN
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Ops and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,Management,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,Executive,NaN
3,10K,Matthew,Lamb,CTO,matt@10kview.com,NaN,NaN,CXO,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,CXO,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,Management,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,Staff,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,Staff,NaN
8,1Password,Emily,Payne,Sales Enablement Mgr.,emily.payne@1password.com,(888) 710-9976,NaN,Management,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Ops",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,Management,NaN


In [778]:
# See how many Contact Levels are still null (~3.4%)
df2.info()

# 123 titles not given Contact Levels

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4532 entries, 0 to 4531
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Account Name   4532 non-null   object 
 1   First Name     4532 non-null   object 
 2   Last Name      4532 non-null   object 
 3   Title          4532 non-null   object 
 4   Email          4532 non-null   object 
 5   Phone          4391 non-null   object 
 6   Mobile         6 non-null      object 
 7   Contact Level  4422 non-null   object 
 8   Department     0 non-null      float64
dtypes: float64(1), object(8)
memory usage: 318.8+ KB


In [779]:
# group remaining rows by Title to see what is being left out of the categorization (0 in 'Contact Level')
contact_check = df2.groupby('Title').count().sort_values('Account Name', ascending=False)
contact_check[contact_check['Contact Level'] == 0].head(20)
# df2[df2['Title']=='Sales Ops Assoc.']

### FIRST PASS
# # significant numbers (10+)
# 'Developer' (LOTS)
# 'Vp' and any uppercase abbreviation (Ceo, Coo, Svp, Evp) (LOTS)
# 'Assoc.' (LOTS)
# 'Sales Ops'
# 'Support'
# 'Engineer'
# 'Chief Revenue Officer'
# 'Planner'

# # less sigificant (1-10)
# 'Integration'
# 'Integration Lead'
# 'Strategist'
# 'Marketer'
# 'Instructor'
# 'Advisor'
# 'Planner'
# 'Mech.' or 'Mechanic'
# 'Oracle'
# 'Trainee'

### SECOND PASS
# lowercase words that are in categories
# 'Management' instead of 'Manager'
# QA or Quality Assurance
# Analytics instead of 'Analyst'
# Planning
# Sales Ops
# Recruiter or Recruitment
# Scrum Master
# Service Operations
# Expert
# Specialist
# Coach

### THIRD PASS
# 'Sales Ops Assoc.' x14
# 'Sales Ops' x13
# lowercase misses x11+
# Salesforce Assoc. x3
# Mechanic x2
# Sr. Assoc. x2

,Account Name,First Name,Last Name,Email,Phone,Mobile,Contact Level,Department
Title,,,,,,,,
Maintenance Mechanic A,3,3,3,3,3,0,0,0
Oracle,3,3,3,3,3,0,0,0
Member,2,2,2,2,2,0,0,0
"Sr. Assoc., M&A Transaction Services",2,2,2,2,2,0,0,0
Администратор Salesforce,2,2,2,2,2,0,0,0
"Assoc. - Venture Capital, Private Equity, and M&A Services",2,2,2,2,2,0,0,0
Salesforce Functional Configurator,1,1,1,1,1,0,0,0
Salesforce Evangelist,1,1,1,1,0,0,0,0
Sales/Revenue Operations and Enablement,1,1,1,1,1,0,0,0


In [780]:
df2.head(50)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Mgr., Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,Management,NaN
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Ops and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,Management,NaN
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,Executive,NaN
3,10K,Matthew,Lamb,CTO,matt@10kview.com,NaN,NaN,CXO,NaN
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,CXO,NaN
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,Management,NaN
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,+1 949-748-5970,NaN,Staff,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,Staff,NaN
8,1Password,Emily,Payne,Sales Enablement Mgr.,emily.payne@1password.com,(888) 710-9976,NaN,Management,NaN
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Ops",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,Management,NaN


In [781]:
# export df2 for excel manual check
# df2.to_excel('Sales_Ops_v2.xlsx')

## 3. Categorize 'Department' Field
Fields:
- Sales Ops
- Sales Support
- Sales Enablement
- Sales
- Finance
- Accounting
- FP&A
- IT
- Legal
- Customer Success
- Consulting
- Marketing
- HR
- Procurement
- FinSys
- Operations

**Top Ten Missed:**
- Salesforce Developer (538+) **IT?**
- Salesforce Administrator (91+)
- Salesforce Mgr./Dir.(30+)
- Planning Mgr. (12+)
- Salesforce/Solution(s) Architect (19+)
- CRO (9)
- COO (4) **Operations?**
- Salesforce Technical Lead (4)
- Salesforce Trainee (4)
- CEO & Founder / Founder and CEO (7)
- Co-Founder (3)
- Maintenance Mechanic (3+) **Operations?**
- President (3)


Double check:
- FP&A (Planning)

Ask LK:
- 'Salesforce Developer' categorization
- 'Salesforce/Solution(s) Architect' categorization
- CRO/COO/CEO/Founder/Co-Founder/President
- 'Salesforce Mgr./Dir.'
- Other 'Support' titles ; Trainee/Mechanic/Instructor/etc.

### (a) Sales Ops – all titles containing:
- Sales Operations
- Sales Ops
- Revenue Operations
- RevOps

In [782]:
# filter dataframe by Sales Ops titles, then set the 'Department' column equal to 'Sales Ops'
# SYNTAX: df.loc[df[‘filter column’] condition, ‘target column name’] = ‘target column value if condition is met’
df2.loc[df2['Title'].str.contains(
                                  'Sales Operations|'
                                  'Sales Ops|'
                                  'Revenue Operations|'
                                  'RevOps'
                                  ,na=False, regex=True),'Department'] = 'Sales Ops'

# df2.head()
df2[df2['Department']=='Sales Ops'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Ops and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,Management,Sales Ops
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Ops",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,Management,Sales Ops
11,2-10 Home Buyers Warranty,Zachary,Ratzlaff,Sales Ops Mgr.,zaratzlaff@2-10.com,(720) 747-6000,NaN,Management,Sales Ops
12,2-10 Home Buyers Warranty,Zach,Ratzlaff,Sales Ops Mgr.,zratzlaff@2-10.com,(720) 747-6000,NaN,Management,Sales Ops
14,3BL Media,Emily,Tuohy,Revenue Operations Mgr.,etuohy@3blmedia.com,(866) 508-0993,NaN,Management,Sales Ops
15,3STEP Sports,Deion,Wilson,Sales Ops Specialist,deion.wilson@threestep.com,(781) 722-0338,NaN,Staff,Sales Ops
22,"8x8, Inc.",Helena,Ostlin,"Sr. Mgr., Sales Ops",helena.ostlin@8x8.com,(844) 716-2409,NaN,Management,Sales Ops
24,A&M Wood Specialty,Raphael,Giuffrida,Sales Ops Team Lead,rgiuffrida@forloversofwood.com,+1 519-653-9322,NaN,Staff,Sales Ops
27,AAA Club Alliance,John,Cccm,Mgr. Insurance Agency Retention Sales Ops,jstassi@aaamidatlantic.com,(800) 222-4357,NaN,Management,Sales Ops
29,Ability Network Inc.,Kelli,Davis,Sr. Mgr. Sales Ops,kelli.davis@inovalon.com,(612) 460-4301,NaN,Management,Sales Ops


### (b) Sales Support
- containing “Sales” and “Support” but not the 4 Sales Ops titles

In [783]:
# create condition for containing both 'Sales' and 'Support' but none of 'Ops, Operation, or Operations'
sales_support = (df2['Title'].str.contains(r"Sales",na=False, regex=True) &\
        df2['Title'].str.contains(r"Support",na=False, regex=True) &\
        ~df2['Title'].str.contains(r"Ops|Operation|Operations",na=False, regex=True))

# apply the condition to filter the rows and then set the 'Department' column to 'Sales Support'
df2.loc[sales_support, 'Department'] = "Sales Support"

# df2.head(50)
df2[df2['Department']=='Sales Support'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
31,ACASS,Shauna,Dunham,Mgr. of Sales Support,sdunham@acass.com,(514) 636-1099,NaN,Management,Sales Support
35,Access Development,Jodi,Lee,Sales Support Admin,jodi.lee@accessdevelopment.com,+1 800-840-0032,NaN,Staff,Sales Support
49,Actian Corporation,Paul,Navarro,Support Sales Mgr.,paul.navarro@actian.com,(650) 587-5500,NaN,Management,Sales Support
58,Acuity International,Jefrey,Joy,Sr. Salesforce Administrator / SF Team Support...,jefrey.joy@acuityinternational.com,(703) 261-1110,NaN,Staff,Sales Support
81,Acxiom LLC,Dana,Ferguson,Sr. Mgr. of Sales Support,dana.blackburn@acxiom.com,(866) 352-3267,NaN,Management,Sales Support
83,"Adelman Travel Group, Inc.",Laura,Schad,"Mgr., Sales Support",lschad@adelmantravel.com,+1 414-410-8300,NaN,Management,Sales Support
89,Advanced Sterilization Products (ASP),Ali,Balahoroglu,Distribution and Sales Support Mgr. Turkey&Aze...,ali.balahoroglu@asp.com,//1-888-783-7723,NaN,Management,Sales Support
90,Advanced Sterilization Products (ASP),Costas,Bastas,Sales Rep / Technical Support,costas.bastas@asp.com,//1-888-783-7723,NaN,Staff,Sales Support
91,Advanced Sterilization Products (ASP),Nikos,Klapadakis,Sales Representative and Technical Support,nikos.klapadakis@asp.com,//1-888-783-7723,NaN,Staff,Sales Support
92,Advanced Sterilization Products (ASP),Patrizia,Scarponi,Sales Force Contractual Support,patrizia.scarponi@asp.com,//1-888-783-7723,NaN,Staff,Sales Support


### (c) Sales Enablement
- containing “Enablement” but NOT the 4 Sales Ops titles

In [784]:
# create condition for containing both 'Enablement' but none of 'Ops, Operation, or Operations'
sales_enable = (df2['Title'].str.contains(r"Enablement",na=False, regex=True) &
        ~df2['Title'].str.contains(r"Ops|Operation|Operations",na=False, regex=True))

# apply the condition to filter the rows and then set the 'Department' column to 'Sales Support'
df2.loc[sales_enable, 'Department'] = "Sales Enablement"


# df2.head(50)
df2[df2['Department']=='Sales Enablement'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Mgr., Sales Enablement",yves.laflamme@1800gotjunk.com,+1 800-468-5865,NaN,Management,Sales Enablement
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,Staff,Sales Enablement
8,1Password,Emily,Payne,Sales Enablement Mgr.,emily.payne@1password.com,(888) 710-9976,NaN,Management,Sales Enablement
23,"8x8, Inc.",Barbara,Phillips,Sales Onboarding & Enablement Mgr.,barbara.phillips@8x8.com,(844) 716-2409,NaN,Management,Sales Enablement
43,ACI Learning,Dylan,Stenulson,Sales Enablement Specialist,dylan.stenulson@acilearning.com,(719) 596-4545,NaN,Staff,Sales Enablement
56,"Activision Blizzard, Inc.",Dana,Dobbie,Global Head of Sales Enablement at Activision ...,dana.dobbie@activision.com,+1-310-255-2000,NaN,Management,Sales Enablement
100,Aerohive Networks,Christi,Nicolacopoulos,"VP, Sales Enablement & Communications",cnicolacopoulos@extremenetworks.com,(408) 510-6100,NaN,Management,Sales Enablement
107,Agility PR Solutions,Kevin,Black,"Mgr., Sales Enablement",kevin.black@agilitypr.com,(866) 545-3745,NaN,Management,Sales Enablement
116,Airtable,Amanda,Dowdakin,"Mgr., Post Sales Enablement",amanda.dowdakin@airtable.com,(415) 200-2040,NaN,Management,Sales Enablement
117,"Akoya Biosciences, Inc.",Laura,Faken,"Assoc. Dir., Commercial Training (Sales Enable...",lfaken@akoyabio.com,(415) 765-6980,NaN,Management,Sales Enablement


### (d) Sales
- NOT Sales Ops, Sales Support or Sales Enablement AND containing:
- Sales 
- Business Development
- Account Exec **(Exec.?)**
- AE
- Account Mgr.
- Sales Development Representative
- SDR
- Sales Development Executive
- SDE
- Business Development Representative
- BDR
- Business Development Mgr.
- BDM
- Channel, Channels

**Notes**:
- should these acronyms be created and added at the beginning?

In [785]:
# create condition for containing any of the Sales keywords,
# but NOT being from the Sales Ops, Sales Support, or Sales Enablement departments
sales = (df2['Title'].str.contains(
                                  'Sales(?!force)|'
                                  'Business Development|'
                                  'Account Exec|'
                                  'AE|'
                                  'Account Mgr\.|'
                                  'Sales Development Representative|'
                                  'SDR|'
                                  'Sales Development Representative|'
                                  'SDE|'
                                  'Business Development Representative|'
                                  'BDR|'
                                  'Business Development Mgr\.|'
                                  'BDM|'
                                  'Channel|Channels'
                                  ,na=False, regex=True) &
                ~df2['Department'].str.contains(r"Sales Ops|Sales Support|Sales Enablement",na=False, regex=True))

# apply the condition to filter the rows and then set the 'Department' column to 'Sales Support'
df2.loc[sales, 'Department'] = "Sales"

# df2.head(50)
df2[df2['Department']=='Sales'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,Management,Sales
16,4Refuel Canada,Victor,Chatellier,Sales and Operations Coordinator,vchatellier@4refuel.com,+1 888-473-3835,NaN,Staff,Sales
17,4Refuel Canada,Goli,Golzari,Sales and Operations Coordinator,ggolzari@4refuel.com,+1 888-473-3835,NaN,Staff,Sales
28,AAGI,Brian,Cook,"Western Sales, VP",brianc@aagi.com,+1(888) 442-2886,NaN,Management,Sales
48,Acra Lending,Jason,Sheridan,"VP, Wholesale Sales",jsheridan@citadelservicing.com,+1(888) 800-7661,NaN,Management,Sales
51,Active International Ltd,Robert,Wilson,SVP Merchandise Sales,robert.wilson@activeinternational.com,+1 845-732-8806,NaN,Management,Sales
53,ActiveWorx,George,Thomas,VP of Sales,gthomas@miriasystems.com,+1 484-446-3300,NaN,Management,Sales
61,Acumatica,John,Cheatham,Inside Sales Mgr. (US and Canada),jcheatham@acumatica.com,+1 888-228-8300,NaN,Management,Sales
65,Acumatica,Louisa,Gooding,Sr. Business Development Mgr.,louisa.gooding@acumatica.com,+1 888-228-8300,NaN,Management,Sales
68,Acumatica,Foy,Mainor,ISV Business Development Mgr.,foy.mainor@acumatica.com,+1 888-228-8300,NaN,Management,Sales


### (e) Finance – all titles containing:
- CFO
- Controller
- Controlling
- Treasury
- Treasurer 
- Auditor
- Auditing
- Finance
- Financial
- Tax
- Investor Relations
- SEC Reporting
- Accounts Payable, AP, A/P 
- Accounts Receivable, AR, A/R
- Investment

In [786]:
# make 'Department' column Finance if 'Title' column contains any financial keywords
df2.loc[df2['Title'].str.contains(
                                  'CFO|'
                                  'Controller|'
                                  'Controlling|'
                                  'Treasury'
                                  'Treasurer|'
                                  'Auditor|'
                                  'Auditing|'
                                  'Finance|'
                                  'Financial|'
                                  'Tax|'
                                  'Investor Relations|'
                                  'SEC Reporting|'
                                  'Accounts Payable|AP|A/P|'
                                  'Accounts Receivable|AR|A/R|'
                                  'Investment'
                                  ,na=False, regex=True),'Department'] = 'Finance'

# df2.head(50)
df2[df2['Department']=='Finance'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
229,AppLovin,Dan,Chau,Dir. of Tax - M&A and Tax Planning,dan.chau@applovin.com,(650) 335-7190,NaN,Management,Finance
693,"CareAR, A Xerox Company",Stephanie,Erdel,"Creative Dir. at CareAR, a Xerox Company",stephanie.erdel@xerox.com,(845) 657-4202,NaN,Management,Finance
780,Charles River Laboratories,Chandrasekhar,Gopisetti,"Principal Specialist, SAP Analytics and Planning",cgopisetti@criver.com,(781) 222-6000,NaN,Staff,Finance
1168,Demoboost,Anna,Decroix,CMO & General Mgr. APAC,a.decroix@demoboost.com,NaN,NaN,CXO,Finance
1229,"Doximity, Inc.",Perry,Gold,"VP, Investor Relations & Revenue Operations",pgold@doximity.com,(650) 200-3901,NaN,Management,Finance
1436,Finastra,Flavia,Siuves,Head of Global Transfer Pricing and Tax Planning,flavia.siuves@finastra.com,+44-20-3320-5000,NaN,Management,Finance
1664,"H&R Block, Inc.",Susan,Davis,Master Tax Advisor,s.davis@dataclaritycorp.com,(919) 256-6711,NaN,Staff,Finance
2069,Kraft Foods,Nathalie,Roggio,Mgr. Support Sales Controlling,nathalie.roggio@kraftheinzcompany.com,NaN,NaN,Management,Finance
2103,LevitJones Holdings LLC,Olga,Levitsky,"Managing Dir., Investment and Product Strategy",oal@levitjones.com,+1(800) 845-4159,NaN,Management,Finance
2231,"Medallia, Inc.",Nader,Hamad,"Sr. Dir. Sales Enablement EMEA, APAC & LATAM (...",nhamad@medallia.com,(650) 321-3000,NaN,Management,Finance


### (f) Accounting
- all titles containing “Accounting” or “Accountant” and NOT any of the Finance titles above

**Notes:**
- no accounting titles at all?

In [787]:
# create condition for containing any of the Accounting keywords,
# but NOT being from Finance department
accounting = (df2['Title'].str.contains(
                                  'Accounting|'
                                  'Accountant'
                                  ,na=False, regex=True) &
        ~df2['Department'].str.contains(r"Finance",na=False, regex=True))

# apply the condition to filter the rows and then set the 'Department' column to 'Accounting'
df2.loc[accounting, 'Department'] = "Accounting"

df2[df2['Department']=='Accounting'].head(10)

## NO ACCOUNTING?? MANUAL CHECK

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department


### (g) FP&A - all titles containing:
- Financial Planning and Analysis
- Financial Planning & Analysis
- Financial Planning, Analysis
- Budgeting
- Planning 
- Forecast
- Forecasting (but not including “Sales Forecasting”)

In [788]:
# make 'Department' column FP&A if 'Title' column contains any financial planning keywords
df2.loc[df2['Title'].str.contains(
                                  'Financial Planning and Analysis|'
                                  'Financial Planning & Analysis|'
                                  'Financial Planning, Analysis|'
                                  'Budgeting|'
                                  'Planning|'
                                  'Forecast|'
                                  '(?<!Sales )Forecasting'
                                  ,na=False, regex=True),'Department'] = 'FP&A'

# df2.head(50)
df2[df2['Department']=='FP&A'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
10,1WorldSync,Renee,Gerami,Dir. of Technology Planning & Delivery,rgerami@1worldsync.com,(866) 280-4013,NaN,Management,FP&A
46,Acquia,Pam,Bragan,"Sr. Dir. of Revenue Programs, Planning and Inc...",pam.bragan@acquia.com,(888) 922-7842,NaN,Management,FP&A
134,"Allegro Microsystems, Inc",Srini,Sekar,Sr. Dir. - Sales Inventory and Operations Plan...,ssekar@allegromicro.com,(508) 853-5000,NaN,Management,FP&A
162,American Media Inc.,Winsford,Belle,Dir. Corp. Planning,wbelle@amilink.com,(212) 545-4800,NaN,Management,FP&A
176,Ampere,John,Lautze,SVP Manufacturing and Planning,jlautze@amperecomputing.com,(669) 770-3700,NaN,Management,FP&A
182,Amylyx Pharmaceuticals,Brett,Rice,"Lead, Inventory Mgr. and Planning",brett_rice@amylyx.com,(617) 571-8872,NaN,Management,FP&A
183,Amylyx Pharmaceuticals,Chriss,Westphal,"Head, Commercial Planning & Analytics",chriss_westphal@amylyx.com,(617) 571-8872,NaN,Management,FP&A
184,"Amyris, Inc.",Kevin,Bussemer,"Channel Planning Mgr., Biossance",bussemer@amyris.com,(510) 450-0761,NaN,Management,FP&A
185,"Amyris, Inc.",Vilma,Decena,Production Planning Mgr.,decena@amyris.com,(510) 450-0761,NaN,Management,FP&A
186,"Amyris, Inc.",Brian,Dencker,"Assoc. Dir., Sales Planning",dencker@amyris.com,(510) 450-0761,NaN,Management,FP&A


### (h) IT - all titles containing: 
- Information Technology
- Information Systems”
- IT (with leading and trailing spaces) **or just word boundaries?**
- IS (with leading and trailing spaces) **or just word boundaries?**
- Systems
- MIS (with leading and trailing spaces)
- Data
- Software
- Application Development
- Help Desk / Helpdesk
- Network
- Project Manager
- Quality Assurance, QA
- BI (with leading and trailing spaces)
- Business Intelligence
- Developer **ADDED**

In [789]:
# make 'Department' column IT if 'Title' column contains any Information Technology keywords
df2.loc[df2['Title'].str.contains(
                                  'Information Technology|'
                                  'Information Systems|'
                                  ' IT |'
                                  ' IS |'
                                  'Systems|'
                                  ' MIS |'
                                  'Data|'
                                  'Software|'
                                  'Application Development|'
                                  'Help Desk / Helpdesk|'
                                  'Network|'
                                  'Project Manager|'
                                  'Quality Assurance, QA|'
                                  ' BI |'
                                  'Business Intelligence|'
                                  'Developer'
                                  ,na=False, regex=True),'Department'] = 'IT'

# df2.head(50)
df2[df2['Department']=='IT'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,Management,IT
21,6Street Digital,Johnson,Tai,Salesforce Developer,johnson.tai@6st.co,+1(833) 242-6439,NaN,Staff,IT
37,Achievers Corp,Mustafa,Avci,"Mgr., Revenue Operations Systems",tugrul.avci@achievers.com,(888) 622-3343,NaN,Management,IT
55,"Activision Blizzard, Inc.",Sam,Akiki,Salesforce Developer,oussama.akiki@activision.com,+1-310-255-2000,NaN,Staff,IT
73,Acumatica,Jake,Poysti,"Mgr., Data Analytics and Visualization",jake.poysti@acumatica.com,+1 888-228-8300,NaN,Management,IT
87,Aduro Advisors,Quan,Dao,Salesforce Solutions Developer,quan@aduroadvisors.com,+1 650 690 2060,NaN,Staff,IT
110,AIMDek Technologies,Ruju,Bhandari,Salesforce Developer,ruju.bhandari@aimdek.com,+91 79654 04042,NaN,Staff,IT
111,AIMDek Technologies,Akriti,Jha,Salesforce Developer,akriti.jha@aimdek.com,+91 79654 04042,NaN,Staff,IT
112,AIMDek Technologies,Krupa,Patel,Salesforce Developer,krupa.patel@aimdek.com,+91 79654 04042,NaN,Staff,IT
113,AIMDek Technologies,Harsh,Shah,Salesforce Developer,harsh.shah@aimdek.com,+91 79654 04042,NaN,Staff,IT


### (i) Legal - all titles containing:
- Attorney
- Counsel
- Esq.

**Notes**:
- 0 rows!

In [790]:
# make 'Department' column Legal if 'Title' column contains any Legal keywords
df2.loc[df2['Title'].str.contains(
                                  'Attorney|'
                                  'Counsel|'
                                  'Esq\.'
                                  ,na=False, regex=True),'Department'] = 'Legal'

# df2.head(50)
df2[df2['Department']=='Legal'].head(10)

## NO LEGAL TITLES AT ALL??

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department


### (j) Customer Success
- Customer Success
- Customer Services
- Account Services
- Customer Care
- Client Services
- Call Center

In [791]:
# make 'Department' column 'Customer Success' if 'Title' column contains any Customer Success keywords
df2.loc[df2['Title'].str.contains(
                                  'Customer Success|'
                                  'Customer Services|'
                                  'Account Services|'
                                  'Customer Care|'
                                  'Client Services|'
                                  'Call Center'
                                  ,na=False, regex=True),'Department'] = 'Customer Success'

# df2.head(50)
df2[df2['Department']=='Customer Success'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
59,Acumatica,Melissa,Bagliano,Customer Success Mgr.,melissa.bagliano@acumatica.com,+1 888-228-8300,NaN,Management,Customer Success
60,Acumatica,Diana,Bowers,Sr. Customer Success Mgr.,diana.bowers@acumatica.com,+1 888-228-8300,NaN,Management,Customer Success
72,Acumatica,Kim,Plank,Customer Success Marketing Mgr.,kim.plank@acumatica.com,+1 888-228-8300,NaN,Management,Customer Success
76,Acumatica,Jacob,Scism,Customer Success Mgr.,jacob.scism@acumatica.com,+1 888-228-8300,NaN,Management,Customer Success
77,Acumatica,Kim,Seiger,Customer Success Mgr.,kim.seiger@acumatica.com,+1 888-228-8300,NaN,Management,Customer Success
78,Acumatica,Sarah,Spangler,Customer Success Mgr.,sarah.spangler@acumatica.com,+1 888-228-8300,NaN,Management,Customer Success
548,Bitwise Industries,John,Palm,VP of Customer Success - Salesforce Practice,jpalm@bitwiseindustries.com,(559) 500-3305,NaN,Management,Customer Success
656,Burlington Medical,Keith,Atkinson,"Dir., Sales Enablement and Customer Success",keith.atkinson@burmed.com,(757) 888-8994,NaN,Management,Customer Success
897,Coca-Cola Canada Bottling Limited,Fil,Mastroiacovo,"Call Center Operations, Sales and Customer Ser...",fmastroiacovo@cokecanada.com,+1 844-383-2653,NaN,Staff,Customer Success
954,Copado,Philip,Cromack,"Team Lead, Customer Success Mgr. at Copado | S...",pcromack@copado.com,+1 415-483-0840,NaN,Management,Customer Success


### (k) Consulting - all titles containing: 
- Consultant
- Consulting
- Practice
- Advisory
- Contractor 

In [792]:
# make 'Department' column 'Consulting' if 'Title' column contains any Consulting keywords
df2.loc[df2['Title'].str.contains(
                                  'Consultant|'
                                  'Consulting|'
                                  'Practice|'
                                  'Advisory|'
                                  'Contractor'
                                  ,na=False, regex=True),'Department'] = 'Consulting'

# df2.head(50)
df2[df2['Department']=='Consulting'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
88,Advanced Network Management,Eduardo,Rodriguez,Dir. of Salesforce and Business Analytics Prac...,eduardo.rodriguez@anm.com,(877) 266-8822,NaN,Management,Consulting
199,Ankura Consulting Group,Alex,Marjanovic,"Sr. Managing Dir. | US West Region, Discovery ...",alex.marjanovic@ankura.com,(214) 200-3695,NaN,Management,Consulting
313,Archima Solutions,Ryan,Raymer,"Managing Dir., Salesforce and Mulesoft Practice",ryan.raymer@archimasolutions.com,(201) 431-7552,NaN,Management,Consulting
346,Ascendix Technologies,Kate,Golovko,Salesforce Practice Coordinator,kholovko@ascendix.com,+1 972-889-8090,NaN,Staff,Consulting
544,Bitwise Industries,Steve,Andersen,"VP of Salesforce Solutions, Salesforce Practice",sandersen@bitwiseindustries.com,(559) 500-3305,NaN,Management,Consulting
545,Bitwise Industries,Tequiliana,Fertil,Dir. of Partnerships - Salesforce Practice,tfertil@bitwiseindustries.com,(559) 500-3305,NaN,Management,Consulting
546,Bitwise Industries,Rachel,Hands,"VP of Services and Delivery, Salesforce Practice",rhands@bitwiseindustries.com,(559) 500-3305,NaN,Management,Consulting
548,Bitwise Industries,John,Palm,VP of Customer Success - Salesforce Practice,jpalm@bitwiseindustries.com,(559) 500-3305,NaN,Management,Consulting
549,Bitwise Industries,Ginger,Whalen,VP Partnerships and Alliances | Salesforce Pra...,gwhalen@bitwiseindustries.com,(559) 500-3305,NaN,Management,Consulting
608,Bottomline Technologies,Rich,Drury,Dir. of Solution Consulting & Sales Enablement,rich.drury@bottomline.com,4.41E+11,NaN,Management,Consulting


### (l) Marketing - all titles containing AND not previously categorized: 
- Marketing
- Brand
- Product Management
- Demand
- Lead Gen
- Media Relations
- eCommerce

In [793]:
# create condition for containing any of the Accounting keywords,
# but NOT being categorized yet (null Department)
marketing = (df2['Title'].str.contains(
                                  'Marketing|'
                                  'Brand|'
                                  'Product Management|'
                                  'Demand|'
                                  'Lead Gen|'
                                  'Media Relations|'
                                  'eCommerce'
                                  ,na=False, regex=True) &
        df2['Department'].isnull())

# apply the condition to filter the rows and then set the 'Department' column to 'Marketing'
df2.loc[marketing, 'Department'] = "Marketing"

df2[df2['Department']=='Marketing'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
66,Acumatica,Dhanish,John,ERP Product Marketing Mgr.,dhanish.john@acumatica.com,+1 888-228-8300,NaN,Management,Marketing
69,Acumatica,James,Mallory,Product Marketing Mgr.,james.mallory@acumatica.com,+1 888-228-8300,NaN,Management,Marketing
74,Acumatica,Tejal,Rastogi,"Marketing Mgr., Demand Generation",tejal.rastogi@acumatica.com,+1 888-228-8300,NaN,Management,Marketing
101,Aeroseal,Donna,Hawke,Strategic Marketing Mgr. - Commercial,donna.hawke@aeroseal.com,+1 937-428-9300,NaN,Management,Marketing
118,"Aledade, Inc.",Lynka,Rivera,Sr. Marketing Campains Ops Specialist,lrivera@aledade.com,+1 844-370-1783,NaN,Staff,Marketing
317,Archstone Consulting,Ileana,Gheorma,"Marketing Dir., Global Demand Generation and A...",igheorma@thehackettgroup.com,(770) 225-3600,NaN,Management,Marketing
391,AudioEye,Chad,Sollis,"SVP, Revenue Marketing & Operations",chad.sollis@audioeye.com,(866) 331-5324,NaN,Management,Marketing
396,"Automation Anywhere, Inc.",Claudia,Michon,SVP of Product and Solutions Marketing. Ex Sal...,claudia.michon@automationanywhere.com,(888) 484-3535,NaN,Management,Marketing
512,Berlitz Corporation,Blanca,Escalera,Salesforce Marketing Cloud Specialist,blanca.alguacil@berlitz.com,+1 609-514-9650,NaN,Staff,Marketing
675,Calm,Ashley,Devincenzo,"Exec. Assistant to the Chief Content Officer, ...",ashley.devincenzo@calm.com,(650) 988-8500,NaN,Management,Marketing


### (m) HR - all titles containing:
- Human Resources
- HR (if not also containing Finance)
- People
- Personnel
- Recruiting
- Compensation
- Benefits
- HRIS

In [794]:
# create condition for containing any of the Accounting keywords,
# but if contains 'HR', not categorized as 'Finance'
hr_no_finance = (df2['Title'].str.contains(r"HR",na=False, regex=True) &
                 ~df2['Title'].str.contains(r"Finance",na=False, regex=True)
                )

hr = (df2['Title'].str.contains(
                                  'Human Resources|'
                                  'People|'
                                  'People|'
                                  'Personnel|'
                                  'Recruiting|'
                                  'Compensation|'
                                  'Benefits'
                                  'HRIS'
                                  ,na=False, regex=True) &
        (hr_no_finance)
     )

# apply the condition to filter the rows and then set the 'Department' column to 'Marketing'
df2.loc[hr, 'Department'] = "HR"

# df2.head(50)
df2[df2['Department']=='HR'].head(100)

# These are as close as I've gotten to the HR without Finance thing using regex, will probably just use conditions
# ((?:[^F ]|F(?!inance))+HR(?:[^F ]|F(?!inance))+)m ## excludes any words starting with 'F' before or after 'HR'
# ^(?!Finance)(?<!Finance )HR(?! Finance)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
4449,Xerox,Florence,Lively,"VP, Human Resources - Xerox/Parc & CHRO, FITTL...",florence.lively@xerox.com,(800) 275-9376,NaN,Management,HR


### (n) Procurement – all titles containing:
- Purchasing
- Procurement

**Note**: 0 procurement titles

In [795]:
# make 'Department' column 'Procurement' if 'Title' column contains any Procurement keywords
df2.loc[df2['Title'].str.contains(
                                  'Purchasing|'
                                  'Procurement'
                                  ,na=False, regex=True),'Department'] = 'Procurement'

# df2.head(50)
df2[df2['Department']=='Procurement'].head(10)

# 0 Procurement titles

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department


### (o) FinSys
- Finance Systems 

**Note**: 0 FinSys titles

In [796]:
# make 'Department' column 'FinSys' if 'Title' column contains any FinSys keywords
df2.loc[df2['Title'].str.contains('Finance Systems',na=False, regex=True),'Department'] = 'FinSys'

# df2.head(50)
df2[df2['Department']=='FinSys'].head(10)

# 0 FinSys Titles

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department


### (p) Operations - all titles containing:
- Operations
- General Mgr.
- GM
- Loan
- Compliance
- M&A
- Acquisitions
- Business Partner
- Business Planning
- Plant
- Production
- Program Mgr.
- Quality
- Innovation
- Research
- Compliance
- Supply Chain
- Underwriter
- Materials
- Grant
- Membership
- Real Estate
- Engineer
- Engineering
- Food
- Design

In [797]:
# make 'Department' column 'Operations' if 'Title' column contains any Operations keywords
df2.loc[df2['Title'].str.contains(
                                  'Operations|'
                                  'General Mgr/.|'
                                  'GM|'
                                  'Loan|'
                                  'Compliance|'
                                  'M&A|'
                                  'Acquisitions|'
                                  'Business Partner|'
                                  'Business Planning|'
                                  'Plant|'
                                  'Production|'
                                  'Program Mgr\.|'
                                  'Quality|'
                                  'Innovation|'
                                  'Research|'
                                  'Compliance|'
                                  'Supply Chain|'
                                  'Underwriter|'
                                  'Materials|'
                                  'Grant|'
                                  'Membership|'
                                  'Real Estate|'
                                  'Engineer|'
                                  'Engineering|'
                                  'Food|'
                                  'Design'
                                  ,na=False, regex=True),'Department'] = 'Operations'

# df2.head(50)
df2[df2['Department']=='Operations'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,Management,Operations
14,3BL Media,Emily,Tuohy,Revenue Operations Mgr.,etuohy@3blmedia.com,(866) 508-0993,NaN,Management,Operations
16,4Refuel Canada,Victor,Chatellier,Sales and Operations Coordinator,vchatellier@4refuel.com,+1 888-473-3835,NaN,Staff,Operations
17,4Refuel Canada,Goli,Golzari,Sales and Operations Coordinator,ggolzari@4refuel.com,+1 888-473-3835,NaN,Staff,Operations
34,Access | Information Management,Cindy,Barrios,Corp. Development Advisor - M&A / Corp. Partne...,cbarrios@accesscorp.com,(973) 808-2885,NaN,Management,Operations
37,Achievers Corp,Mustafa,Avci,"Mgr., Revenue Operations Systems",tugrul.avci@achievers.com,(888) 622-3343,NaN,Management,Operations
42,ACI Learning,William,Mathias,"Global Head, Sales Enablement Services and Rev...",bill.mathias@acilearning.com,(719) 596-4545,NaN,Management,Operations
46,Acquia,Pam,Bragan,"Sr. Dir. of Revenue Programs, Planning and Inc...",pam.bragan@acquia.com,(888) 922-7842,NaN,Management,Operations
47,Acquia,Elizabeth,Levasseur,"Sr. Mgr., Revenue Operations, Programs",elizabeth.levasseur@acquia.com,(888) 922-7842,NaN,Management,Operations
52,ActiveProspect,Shannon,Wiggen,Dir. of Revenue Operations,shannon.wiggen@activeprospect.com,(888) 624-4159,NaN,Management,Operations


### (q) Executive
- CFO
- COO
- CAO
- CSO
- CRO
- CMO
- CTO
- CEO 
- President
- Founder
- Co-founder
- Owner

AND not already being in a Department

- ?titles containing Board, Chair?



In [798]:
# create condition for containing any of the Executive keywords,
# but NOT being categorized yet (null Department)
executive = (df2['Title'].str.contains(
                                  'CFO|'
                                  'COO|'
                                  'CAO|'
                                  'CSO|'
                                  'CRO|'
                                  'CMO|'
                                  'CTO|'
                                  'CEO|'
                                  'President|'
                                  'Founder|'
                                  'Co-[Ff]ounder|'
                                  '^Owner'
                                  ,na=False, regex=True) &
        df2['Department'].isnull()
            )

# apply the condition to filter the rows and then set the 'Department' column to 'Marketing'
df2.loc[executive, 'Department'] = "Executive"

df2[df2['Department']=='Executive'].head(10)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,NaN,NaN,Executive,Executive
3,10K,Matthew,Lamb,CTO,matt@10kview.com,NaN,NaN,CXO,Executive
4,10K,Jared,Miller,COO,jared@10kview.com,NaN,NaN,CXO,Executive
13,3 Creeks Technologies LLC,Scott,Aker,Owner/Founder,scott@3creekstech.com,NaN,NaN,Executive,Executive
18,5P Consulting,Randy,Jasinski,President / Partner,rjasinski@5pconsulting.biz,+1 760-652-9127,NaN,Executive,Executive
19,5P Consulting,Y'vonne,Sisco,Founder / CEO,yormond@5pconsulting.biz,+1 760-652-9127,NaN,Executive,Executive
93,AdvancedSales.ai,Greg,Plum,CMO,greg@plumuc.com,302.530.5654,NaN,CXO,Executive
105,Agency73,Stryder,Thompkins,Co-Founder and CTO,stryder@agency73.com,+1 858-449-1499,NaN,Executive,Executive
106,Agile Cloud Consulting,Sharif,Shaalan,CEO & Founder,sharif.shaalan@agilecloudconsulting.com,+18882445350,NaN,Executive,Executive
321,Argano,Ron,Guadagno,Ex-CRO (CRO),rguadagno@interrel.com,+1 972-735-8716,NaN,CXO,Executive


### Check for null 'Department' rows

In [799]:
df2.info()

# 532 null 'Department rows'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4532 entries, 0 to 4531
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Account Name   4532 non-null   object
 1   First Name     4532 non-null   object
 2   Last Name      4532 non-null   object
 3   Title          4532 non-null   object
 4   Email          4532 non-null   object
 5   Phone          4391 non-null   object
 6   Mobile         6 non-null      object
 7   Contact Level  4422 non-null   object
 8   Department     4068 non-null   object
dtypes: object(9)
memory usage: 318.8+ KB


### Investigate null 'Department' rows

In [800]:
dept_check = df2.groupby('Title').count().sort_values('Account Name', ascending=False)
dept_check[dept_check['Department'] == 0].head(50)

,Account Name,First Name,Last Name,Email,Phone,Mobile,Contact Level,Department
Title,,,,,,,,
Salesforce Administrator,79,79,79,79,68,0,79,0
Sr. Salesforce Administrator,12,12,12,12,12,0,12,0
Salesforce Architect,6,6,6,6,5,0,6,0
Salesforce Solution Architect,5,5,5,5,5,0,5,0
Salesforce Product Mgr.,5,5,5,5,4,0,5,0
Solutions Architect,4,4,4,4,4,0,4,0
Salesforce Technical Lead,4,4,4,4,4,0,4,0
Salesforce Trainee,4,4,4,4,2,0,4,0
Salesforce Technical Architect,4,4,4,4,3,0,4,0


**Missed**: 
- Salesforce (System/Certified) Administrator (100+)
- Salesforce/Solution(s) Architect (40+)
- Recruiter (15+)
- Product Mgr.
- Maintenance Mechanic
- Member
- Sr. Mgr., Salesforce
- Oracle
- VP, Head of FINS & Digital Experience
- Assoc. Enterprise Architect - Salesforce	
- CRM Mgr.

- Salesforce (Platform) Mgr./Dir./Assoc.(30+)
- Salesforce Lead Admin	
- Salesforce Geek, MVP
- Salesforce Functional Configurator
- Salesforce Lead
- Salesforce Evangelist
- Salesforce Technical Lead
- Salesforce Trainee
- Salesforce Programme Mgr.
- Salesforce QA
- Salesforce Business Solution Expert
- Salesforce Engagement Mgr.
- Salesforce Delivery Mgr.

**Fixed**
Salesforce Developer (538+) **-> IT**

Ideas:
- add Developers to IT
- 'Salesforce Admin' department for all the random Salesforce 'Mgr./Lead/Geek/Configurator/Admin/Trainee' titles?


## 4. Transform Phone Numbers
- for both Contacts and Companies:
    - remove “+1” from all phone numbers (including the trailing space)
    
**Notes**: 
- some phone numbers in XXX-XXX-XXXX format; want them transformed?

In [801]:
# '\+1 \([\d]+\) [\d]+-[\d]+ ### FOR FULL PHONE NUMBER (UNNEEDED)

# remove any '+1's from the 'Phone' column, with or without a trailing space
df2['Phone'] = df2['Phone'].astype(str).apply(lambda x: re.sub(r"\+1 ?", "", x))

# df2['Phone'].head(50)
df2[df2['Phone'].astype(str).str.contains(r"\+1 ?")].head(10)



,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department


## 5. Transform Zip Codes

- Make sure Zip Code column formatted as such (to not lose leading zeroes)

- No Zip Code column?

In [802]:
df2.head(50)

,Account Name,First Name,Last Name,Title,Email,Phone,Mobile,Contact Level,Department
0,1-800-GOT-JUNK?,Yves,Laflamme,"Mgr., Sales Enablement",yves.laflamme@1800gotjunk.com,800-468-5865,NaN,Management,Sales Enablement
1,100 Thieves,Kelsey,Schultz,"Dir., Head of Sales Ops and Activations",k.schultz@100thieves.com,(424) 298-0196,NaN,Management,Sales Ops
2,10K,Nick,Hamm,Founder and CEO,nick@10kview.com,nan,NaN,Executive,Executive
3,10K,Matthew,Lamb,CTO,matt@10kview.com,nan,NaN,CXO,Executive
4,10K,Jared,Miller,COO,jared@10kview.com,nan,NaN,CXO,Executive
5,160 Driving Academy,Rob,Dearth,"Head of Contact Center Operations, Truckers Ne...",rdearth@160drivingacademy.com,(877) 350-0884,NaN,Management,Operations
6,174 Power Global,David,Park,Salesforce Administrator,david.park@174powerglobal.com,949-748-5970,NaN,Staff,NaN
7,1Password,Jessica,Coscarelli,"Team Lead, Sales Enablement",jessica.coscarelli@agilebits.com,(888) 710-9976,NaN,Staff,Sales Enablement
8,1Password,Emily,Payne,Sales Enablement Mgr.,emily.payne@1password.com,(888) 710-9976,NaN,Management,Sales Enablement
9,1Stdibs.Com Inc,Charlotte,Noordsy,"Mgr., Sales Ops",charlotte.noordsy@1stdibs.com,(877) 721-3427,NaN,Management,Sales Ops


## Final Export to Excel File

In [803]:
# export df2 for excel manual check
df2.to_excel('Sales_Ops_v3.xlsx')